In [1]:
# import ee and folium
import ee
import folium

In [2]:
#authentificate, generate token and allow access rights
ee.Authenticate()

True

In [3]:
ee.Initialize()

## Setting the area of interest

In [4]:
# area of interest in salzburg, austria
aoi = ee.Geometry.Rectangle(12.7,
                            46,
                            13.3,
                            48.9)

## Setting the time Period

In [5]:
start_date = '2024-01-01'
end_date = '2024-04-30'

## Getting the Data

In [6]:
# select sentinel 2 imagery
s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(aoi).filterDate(start_date, end_date)
s2_collection = s2_collection.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20);

# amount of images in the selection
print(s2_collection.size().getInfo())

# reduce collection to mean
mean_collection = s2_collection.reduce(ee.Reducer.mean())

C:\Users\DELL\anaconda3\envs\A3_Asad\Lib\site-packages\ee\deprecation.py:202: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


76


## Calculating NDVI

In [7]:
# Function to calculate the NDVI
def calculate_ndvi(image):
    ndvi = image.expression(
        '(NIR - RED) / (NIR + RED)',
        {
            'NIR': image.select('B8_mean'),
            'RED': image.select('B4_mean')
        })
    return ndvi.rename('NDVI')

In [8]:
# Calculate the NDVI
ndvi_mean = calculate_ndvi(mean_collection)

# Clip it to the AOI
ndvi_mean = ndvi_mean.clip(aoi)

In [9]:
ndvi_mean.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [2, 5],
   'origin': [12, 45],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[12.7, 46],
     [13.3, 46],
     [13.3, 48.9],
     [12.7, 48.9],
     [12.7, 46]]]}}}

## Map the Results

In [10]:
# function to add the layer to folium map
def add_ee_layer(self, ee_object, color, name):
    map_id_dict = ee.Image(ee_object).getMapId(color)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data GEE",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# add the earth engine function to the folium map
folium.Map.add_ee_layer = add_ee_layer

# create folium map
map = folium.Map(location=[47.80, 13.01], zoom_start = 12)

# setting the vis variables
ndvi_color = {
    'min': -1,
    'max': 1.0,
    'palette': ['blue', 'white', 'green']
}

# add the ndvi layer to the map
map.add_ee_layer(ndvi_mean, ndvi_color, 'NDVI')

# display the map
display(map)